# Use BoTorch for feasibility weighted acquisition function

import libearies

In [1]:
import pandas as pd
import numpy as np
import torch
import botorch
import gpytorch
import matplotlib.pyplot as plt

#modules for regression
from botorch.models.gp_regression import SingleTaskGP
from botorch.models.model_list_gp_regression import ModelListGP
from botorch.models.transforms.outcome import Standardize
from gpytorch.mlls.sum_marginal_log_likelihood import SumMarginalLogLikelihood
from gpytorch.mlls.sum_marginal_log_likelihood import ExactMarginalLogLikelihood
from botorch.utils.transforms import unnormalize, normalize

#modules for BO
from botorch.optim.optimize import optimize_acqf, optimize_acqf_list
from botorch.acquisition.objective import GenericMCObjective
from botorch.utils.multi_objective.scalarization import get_chebyshev_scalarization
from botorch.utils.multi_objective.box_decompositions.non_dominated import (
    FastNondominatedPartitioning,
)
from botorch.acquisition.multi_objective.monte_carlo import (
    qExpectedHypervolumeImprovement,
    qNoisyExpectedHypervolumeImprovement,
)
from botorch.utils.sampling import sample_simplex
from botorch import fit_gpytorch_mll
from botorch.exceptions import BadInitialCandidatesWarning
from botorch.sampling.normal import SobolQMCNormalSampler
from botorch.utils.multi_objective.box_decompositions.dominated import (
    DominatedPartitioning,
)
from botorch.utils.multi_objective.pareto import is_non_dominated

from gpytorch.likelihoods import DirichletClassificationLikelihood
from botorch.acquisition.multi_objective.objective import IdentityMCMultiOutputObjective


/Users/gouravkumbhojkar/miniconda3/envs/botorch/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Read and normalize data

In [2]:
filename = r'../data/olhs_combine.xlsx'
x_pd = pd.read_excel(filename, sheet_name='Design', header=[0,1], index_col=[0])
y_pd = pd.read_excel(filename, sheet_name='bo_data', header=[0,1], index_col=[0])

dtype=torch.double

objective_properties = ['Polymer Solubility', 'Gelation Enthalpy', 'Shear Modulus']

x = torch.tensor(x_pd.values, dtype=dtype)
y = torch.tensor(y_pd[objective_properties].values, dtype=dtype)
mfg = torch.tensor(y_pd['Manufacturability'].values, dtype=torch.long)

x_bounds = np.array([[2000, 10000], [0, 100], [0, 40], [5000, 15000], [80, 100], [0,100], [60, 100], [70, 100]])
x_bounds = torch.tensor(x_bounds.T, dtype=dtype)

x = normalize(x, bounds=x_bounds)

Helper functions

Define and initialize regression and classification model

In [22]:
# Regression model
models = []
for i in range(len(objective_properties)):
    models.append(SingleTaskGP(x, y[:, i].unsqueeze(-1), 
                               outcome_transform=Standardize(m=1)))

# transformation of mfg labels
tmp_likl = DirichletClassificationLikelihood(targets=mfg.squeeze(), alpha_epsilon=1e-4, 
                                             learn_additional_noise=False)
cls_model = SingleTaskGP(train_X=x, train_Y=-tmp_likl.transformed_targets.T.double(), 
                         outcome_transform=Standardize(m=2))

models.append(cls_model)
# mll_cls = ExactMarginalLogLikelihood(cls_model.likelihood, cls_model)

model = ModelListGP(*models)
mll = SumMarginalLogLikelihood(model.likelihood, model)
mll = mll.to(x)

Fit the models to data

In [23]:
# regression model
fit_gpytorch_mll(mll)

SumMarginalLogLikelihood(
  (likelihood): LikelihoodList(
    (likelihoods): ModuleList(
      (0-3): 4 x GaussianLikelihood(
        (noise_covar): HomoskedasticNoise(
          (noise_prior): GammaPrior()
          (raw_noise_constraint): GreaterThan(1.000E-04)
        )
      )
    )
  )
  (model): ModelListGP(
    (models): ModuleList(
      (0-3): 4 x SingleTaskGP(
        (likelihood): GaussianLikelihood(
          (noise_covar): HomoskedasticNoise(
            (noise_prior): GammaPrior()
            (raw_noise_constraint): GreaterThan(1.000E-04)
          )
        )
        (mean_module): ConstantMean()
        (covar_module): ScaleKernel(
          (base_kernel): MaternKernel(
            (lengthscale_prior): GammaPrior()
            (raw_lengthscale_constraint): Positive()
          )
          (outputscale_prior): GammaPrior()
          (raw_outputscale_constraint): Positive()
        )
        (outcome_transform): Standardize()
      )
    )
    (likelihood): LikelihoodList

Prediction for sanity check

In [24]:
model.eval()

with torch.no_grad():
    posterior = model.posterior(x)
    pred_mean = posterior.mean
pred_mean

tensor([[6.0338e+01, 3.4804e+00, 5.7093e+00, 1.3639e+01, 5.2273e-01],
        [3.6136e+01, 5.2524e+01, 2.3242e+00, 1.1603e+00, 1.3002e+01],
        [4.6839e+01, 5.4779e+00, 2.7008e+00, 1.2017e+00, 1.2960e+01],
        [4.1833e+01, 4.7513e+00, 2.5480e+00, 1.0066e+00, 1.3155e+01],
        [1.2645e+02, 2.3166e+00, 3.2931e+00, 1.3694e+01, 4.6791e-01],
        [9.1096e+01, 8.0907e+01, 2.8848e+00, 1.3672e+01, 4.9006e-01],
        [9.4297e+01, 2.7102e+00, 3.7552e+00, 1.3590e+01, 5.7191e-01],
        [7.5021e+01, 2.6904e+01, 4.7946e+00, 1.3638e+01, 5.2376e-01],
        [5.7483e+01, 1.0567e+02, 5.8902e+00, 1.3650e+01, 5.1176e-01],
        [1.7562e+02, 3.9078e+00, 1.0532e+01, 1.3692e+01, 4.6987e-01],
        [5.7746e+01, 1.1100e+02, 1.6696e+00, 1.3527e+01, 6.3490e-01],
        [1.0777e+02, 1.7486e+01, 3.3631e+00, 1.3655e+01, 5.0665e-01],
        [8.7002e+01, 9.2771e+00, 1.0524e+01, 1.3645e+01, 5.1683e-01],
        [7.3058e+01, 1.0360e+02, 3.8432e+01, 1.3589e+01, 5.7306e-01],
        [5.4783e+01,

In [13]:
# cls_model.eval()

# with torch.no_grad():
#     posterior = cls_model.posterior(x)
# samples = posterior.sample(torch.Size((256,))).exp()
# probabilities = (samples / samples.sum(-1, keepdim=True)).mean(0)
# probabilities[:, 0] # 1st col gives probability of manufacturability

tensor([9.9998e-01, 4.5079e-05, 6.1726e-05, 3.3730e-05, 9.9999e-01, 9.9997e-01,
        9.9999e-01, 9.9999e-01, 9.9999e-01, 9.9998e-01, 9.9998e-01, 9.9999e-01,
        9.9998e-01, 9.9999e-01, 4.5224e-05, 9.9998e-01, 9.9999e-01, 3.7471e-05,
        9.9999e-01, 5.9146e-05, 5.0092e-05, 9.9998e-01, 9.9998e-01, 9.9999e-01,
        9.9998e-01, 9.9997e-01, 9.9998e-01, 9.9999e-01, 9.9999e-01, 9.9998e-01,
        9.9999e-01, 9.9998e-01, 9.9999e-01, 3.5988e-05, 9.9999e-01, 9.9998e-01,
        9.9998e-01, 9.9999e-01, 9.9999e-01, 9.9996e-01], dtype=torch.float64)

Helper functions for BO

In [17]:
bounds = torch.zeros(2, 8)
bounds[1] = 1

BATCH_SIZE = 4      # Number of candidates selected in each BO run/iteration
NUM_RESTARTS = 10   # Restarts during BO run
RAW_SAMPLES = 512   

ref_point = ref_point = torch.tensor([18, 0.1, 0.01], dtype=dtype)

In [25]:
from typing import Optional

def obj_callable(Z: torch.Tensor, X: Optional[torch.Tensor] = None):
    return IdentityMCMultiOutputObjective([0,1,2])

def constraint_callable(Z):
    class0 = Z[..., -2]
    class1 = Z[..., -1]
    return class1 - class0 #neg for feasiblle values

In [28]:
def optimize_qnehvi_and_get_observation(model, train_x, train_obj, sampler):
    """Optimizes the qEHVI acquisition function, and returns a new candidate and observation."""
    with torch.no_grad():
        pred = model.posterior(normalize(train_x, x_bounds)).mean
    
    # partitioning = FastNondominatedPartitioning(
    #     ref_point= ref_point,
    #     Y=pred,
    # )

    acq_func = qNoisyExpectedHypervolumeImprovement(
        model=model,
        ref_point=ref_point,
        X_baseline=train_x,
        sampler=sampler,
        prune_baseline=True,
        objective=IdentityMCMultiOutputObjective(outcomes=[0, 1, 2]),
        constraints=[constraint_callable],
    )

    #optimize
    candidates, _ = optimize_acqf(
        acq_function=acq_func,
        bounds=bounds,
        q=BATCH_SIZE,
        num_restarts=NUM_RESTARTS,
        raw_samples=RAW_SAMPLES,
        options={"batch_limit":5, "maxiter": 200},
        sequential=True,
    )

    new_x = unnormalize(candidates.detach(), bounds=x_bounds)
    return new_x
    

In [29]:
sampler = SobolQMCNormalSampler(sample_shape=torch.Size([128]))

new_x_qnehvi = optimize_qnehvi_and_get_observation(model, x, y, sampler)

In [30]:
new_x_qnehvi

tensor([[4.4049e+03, 6.7979e+01, 2.9767e+01, 1.5000e+04, 9.8777e+01, 3.4605e+00,
         8.6779e+01, 8.7559e+01],
        [4.5762e+03, 4.8585e+01, 2.2482e+01, 1.4433e+04, 9.1648e+01, 7.5514e+01,
         9.2885e+01, 8.5352e+01],
        [4.8585e+03, 4.5736e+01, 2.5860e+01, 1.4606e+04, 9.4307e+01, 8.1446e+01,
         9.1930e+01, 9.1791e+01],
        [3.8224e+03, 6.3609e+01, 1.8460e+01, 1.4202e+04, 8.7815e+01, 7.4481e+01,
         9.4315e+01, 7.7646e+01]], dtype=torch.float64)

In [34]:
model.posterior(new_x_qnehvi).mean

tensor([[148.9710,  22.8098,  13.7619,  11.1271,   3.0349],
        [148.9710,  22.8098,  13.7619,  11.1271,   3.0349],
        [148.9710,  22.8098,  13.7619,  11.1271,   3.0349],
        [148.9710,  22.8098,  13.7619,  11.1271,   3.0349]],
       dtype=torch.float64, grad_fn=<CloneBackward0>)

In [35]:
cls_likl = DirichletClassificationLikelihood(targets=mfg.squeeze(), alpha_epsilon=1e-4, learn_additional_noise=False)
cls_model = SingleTaskGP(train_X=x, train_Y=cls_likl.transformed_targets.T.double(), outcome_transform=Standardize(m=2))
mll_cls = ExactMarginalLogLikelihood(cls_model.likelihood, cls_model)
fit_gpytorch_mll(mll_cls)

cls_model.eval()
with torch.no_grad():
    post_cls = cls_model.posterior(new_x_qnehvi)
samples_cls = post_cls.sample(torch.Size((256,))).exp()
probabilities = (samples_cls / samples_cls.sum(-1, keepdim=True)).mean(0)
probabilities

tensor([[0.1713, 0.8287],
        [0.1863, 0.8137],
        [0.1441, 0.8559],
        [0.1521, 0.8479]], dtype=torch.float64)